<a href="https://colab.research.google.com/github/RajAakash/DataParallelism/blob/main/DeepSpeed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install deepspeed

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.0 MB/s eta 0:00:00
  Created wheel for deepspeed: filename=deepspeed-0.13.3-py3-none-any.whl size=1365810 sha256=fc536d85a075a7b9c52523357a31483ed5fa427e55c5923dbf8b2c488c2ea38a
  Stored in directory: /root/.cache/pip/wheels/a7/38/4d/1d76c713dd7259a7c42f619cd91944528b76c3ede423ed674a
Successfully built deepspeed


In [2]:
pip install mpi4py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 8.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.5-cp310-cp310-linux_x86_64.whl size=2746532 sha256=d5b18a1ec599768d4d068b25ed1ad85a201e96434097511ae30006a2da4440e2
  Stored in directory: /root/.cache/pip/wheels/18/2b/7f/c852523089e9182b45fca50ff56f49a51eeb6284fd25a66713
Successfully built mpi4py


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import deepspeed

[2024-02-26 00:27:25,203] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [4]:
import torchvision
import torchvision.transforms as transforms
import deepspeed
import mpi4py

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize((0.5,), (0.5,))  # Normalize the dataset
])

# Load MNIST training data
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

# Load MNIST test data
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

100%|██████████| 9912422/9912422 [00:00<00:00, 107724445.83it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 20434496.26it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 33445002.50it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw




100%|██████████| 4542/4542 [00:00<00:00, 5143231.31it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [6]:
class MNISTModel(nn.Module):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [11]:
model = MNISTModel()

# Define an optimizer. Note: Do not initialize the optimizer parameters yet.
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [12]:
# DeepSpeed configuration
deepspeed_config = {
    "train_batch_size": 64,
    "gradient_accumulation_steps": 1,
    "fp16": {
        "enabled": False
    },
    # Add other DeepSpeed config options as needed.
}

model_engine, optimizer, _, _ = deepspeed.initialize(model=model,
                                                     optimizer=optimizer,  # Pass the optimizer here
                                                     config=deepspeed_config)

[2024-02-26 00:34:25,616] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed info: version=0.13.3, git-hash=unknown, git-branch=unknown
[2024-02-26 00:34:25,640] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Flops Profiler Enabled: False
[2024-02-26 00:34:25,646] [INFO] [logging.py:96:log_dist] [Rank 0] Using client Optimizer as basic optimizer
[2024-02-26 00:34:25,650] [INFO] [logging.py:96:log_dist] [Rank 0] Removing param_group that has no 'params' in the basic Optimizer
[2024-02-26 00:34:25,656] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Basic Optimizer = Adam
[2024-02-26 00:34:25,661] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed Final Optimizer = Adam
[2024-02-26 00:34:25,663] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed using client LR scheduler
[2024-02-26 00:34:25,668] [INFO] [logging.py:96:log_dist] [Rank 0] DeepSpeed LR Scheduler = None
[2024-02-26 00:34:25,672] [INFO] [logging.py:96:log_dist] [Rank 0] step=0, skipped=0, lr=[0.001], mom=[(0.9, 0.999)]

In [13]:
for epoch in range(5):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(model_engine.local_rank), target.to(model_engine.local_rank)
        data, target = data.float(), target.long()  # Ensure data types match expected
        optimizer.zero_grad()
        output = model_engine(data)
        loss = nn.CrossEntropyLoss()(output, target)
        model_engine.backward(loss)
        model_engine.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

[2024-02-26 00:34:43,027] [INFO] [logging.py:96:log_dist] [Rank 0] step=10, skipped=0, lr=[0.001], mom=[(0.9, 0.999)]
[2024-02-26 00:34:43,043] [INFO] [timer.py:260:stop] epoch=0/micro_step=10/global_step=10, RunningAvgSamplesPerSec=4554.114635201507, CurrSamplesPerSec=2191.846623662938, MemAllocated=0.03GB, MaxMemAllocated=0.03GB
[2024-02-26 00:34:43,560] [INFO] [logging.py:96:log_dist] [Rank 0] step=20, skipped=0, lr=[0.001], mom=[(0.9, 0.999)]
[2024-02-26 00:34:43,563] [INFO] [timer.py:260:stop] epoch=0/micro_step=20/global_step=20, RunningAvgSamplesPerSec=6685.009841073511, CurrSamplesPerSec=11665.020684860074, MemAllocated=0.03GB, MaxMemAllocated=0.03GB
[2024-02-26 00:34:44,085] [INFO] [logging.py:96:log_dist] [Rank 0] step=30, skipped=0, lr=[0.001], mom=[(0.9, 0.999)]
[2024-02-26 00:34:44,090] [INFO] [timer.py:260:stop] epoch=0/micro_step=30/global_step=30, RunningAvgSamplesPerSec=6677.184503572594, CurrSamplesPerSec=8339.353692255117, MemAllocated=0.03GB, MaxMemAllocated=0.03GB
